In [26]:
import random
import cProfile

# Radix sort

In [37]:
def is_sorted(array):
    for i in xrange(1, len(array)):
        if array[i-1] > array[i]:
            return False
    return True

def radix_sort(array, b):
    assert b > 1
    i = 0
    while True:
        if is_sorted(array):
            break
        
        buckets = [ [] for _ in xrange(b)]
        for num in array:
            bucket_idx = (num / b**i) % b
            buckets[bucket_idx].append(num)
        next_index = 0
        for bucket in buckets:
            for num in bucket:
                array[next_index] = num
                next_index += 1
        
        i += 1


### Verify that it works on a simple example

In [2]:
example = [5,3,2,5,6,1,2,4]
radix_sort(example, 2)
example

[1, 2, 2, 3, 4, 5, 5, 6]

### We are now moving on to bigger examples
The line
```python
random.seed(1)
```
ensures that we always generate the same example given test size (for fairness

In [27]:
def generate_test(test_size):
    random.seed(1)
    example = [ random.randint(0,2**30) for _ in range(test_size)]
    return example

Let's try it for radix sort on different bases.

In [68]:
# BASE 2
example = generate_test(1000000)
cProfile.run("radix_sort(example, 2)")
assert is_sorted(example)

         32000102 function calls in 23.508 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       33    0.848    0.026    3.536    0.107 <ipython-input-49-aec844174429>:1(is_sorted)
        1   18.637   18.637   23.496   23.496 <ipython-input-49-aec844174429>:7(radix_sort)
        1    0.012    0.012   23.508   23.508 <string>:1(<module>)
       33    0.000    0.000    0.000    0.000 {len}
 32000000    1.322    0.000    1.322    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
       33    2.688    0.081    2.688    0.081 {range}




In [41]:
# BASE 16
example = generate_test(1000000)
cProfile.run("radix_sort(example, 2**16)")
assert is_sorted(example)

         2000009 function calls in 1.655 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.170    0.057    0.170    0.057 <ipython-input-37-6c262b17e721>:1(is_sorted)
        1    1.309    1.309    1.631    1.631 <ipython-input-37-6c262b17e721>:7(radix_sort)
        1    0.024    0.024    1.655    1.655 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 {len}
  2000000    0.152    0.000    0.152    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




Intuitively it makes sense. We only need two iterators for $b=16$, while we need 16 for $b=2$. We cannot really go to single iterators, as $b=32$ requires tons of space. 

Now let's try to run the standard sorting algorithm that implemented by Python (hybrid of insertion sort and quicksort). It is a comparison based sorting and is therefore $O(n\ lg\ n)$. Radix sort is $O(n)$. We therefore expect our code to be faster.

In [19]:
example = generate_test(1000000)
cProfile.run("example.sort()")
assert is_sorted(example)

         3 function calls in 0.592 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.592    0.592 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.592    0.592    0.592    0.592 {method 'sort' of 'list' objects}




In reality our code is 3 times slower. What a shame! Let's not give up yet...

### A bit of bit magic.

One of basic operations for radix sort is divide and modulo. In particular if our base is $b$.

Then in every i-th iteration of radix sort algorithm we use as one of the most common operations.

```python
# determine appropriate bucket.
(num / b**i) % b
```

This is great, but it consists of of expensive modulo and division operations (the can take up multiple processor cycles). 

Let's assume that b is a power of 2, i.e. $b=2^k$. Notice that `num / b**i` is equivalent to `num >> (k * i)`. In order to understand why this is the case first convince yourself that division by 2 is equivalent to shaving off rightmost bit.

In [31]:
print(100 / 16, 100 >> 4)
print(128 / 16, 128 >> 4)

(6, 6)
(8, 8)


Similarly notice that for $b=2^k$ we have `num % b` equivalent to `num & (b-1)`. To understand that notice that k lowest bits of `num` correspond to the reminder mod $b$.

In [33]:
print(100 % 16, 100 & 15)
print(128 % 16, 128 & 15)
print(11 % 16, 11 & 15)

(4, 4)
(0, 0)
(11, 11)


Both `&` and `>>` are very efficient and only take on processor cycle. We can augment implementation of radix sort from above to use them instead of `%` and `/`.

In [34]:
def fast_radix_sort(array, k):
    """Fast radix sort with base 2**k.
    
    This implementation uses bitwise operations"""
    assert k > 0
    i = 0
    
    b=2**k
    b_m1 = b - 1
    
    while True:
        if is_sorted(array):
            break
        shift = k * i
        buckets = [ [] for _ in xrange(b)]
        for num in array:
            bucket_idx = (num >> shift) & b_m1
            buckets[bucket_idx].append(num)
                        
        next_index = 0
        for bucket in buckets:
            for num in bucket:
                array[next_index] = num
                next_index += 1
        
        i += 1

In [38]:
example = [5,3,2,5,6,1,2,4]
fast_radix_sort(example, 16)
example

[1, 2, 2, 3, 4, 5, 5, 6]

In [40]:
example = generate_test(1000000)
cProfile.run("fast_radix_sort(example, 16)")
assert is_sorted(example)

         2000009 function calls in 1.463 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.137    1.137    1.443    1.443 <ipython-input-34-2dc028b9f7cf>:1(fast_radix_sort)
        3    0.156    0.052    0.156    0.052 <ipython-input-37-6c262b17e721>:1(is_sorted)
        1    0.020    0.020    1.463    1.463 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 {len}
  2000000    0.151    0.000    0.151    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




*Good news* We got 0.2s speed up Yay!

*Bad news* We are still nowhere near the performance of `.sort`. Why is that?

The answer is in our choice of programming language. Python is interpreted and is known to have slowdowns up to 100x compared to low level langauges like C/C++. That's why many of Python routines are secretly implemented in C, `.sort` being one of them. In order to make the comparison fair we should also be allowed to write our implementation in C. Thankfully there's a Cython python extension, which makes it easy to interface with Python and compalies python-like code to C++. It should be pretty straightforward, but don't worry if you don't understand the details of the implementation below.

In [1]:
%load_ext Cython

In [53]:
%%cython --cplus

from libcpp.vector cimport vector
cimport cython


cdef c_is_sorted(list array):
    """Equivalent to implementation of is_sorted from above.
    
    However this one is compiled to pure C++ (thanks to cdef).
    We cannot call it directly.
    
    The reasons it is slightly different from above is the fast that
    
        for current_num in array
        
    Is super-efficient in Cython."""
    cdef unsigned int lastnum = 0
    cdef bint         first_iter = True
    
    for current_num in array:
        if not first_iter:
            if lastnum > current_num:
                return False
        else:
            first_iter = False
        lastnum = current_num
    return True

def c_radix_sort(list array, int k):
    assert k > 0
    # Just like in C Cython requires us 
    # to forward declare the variables
    cdef int i          = 0
    cdef int b          = 2 ** k
    cdef int b_m1       = b - 1
    cdef int shift      = 0
    cdef int next_index = 0
    cdef int num        = 0
    # vector[vector[int]] is a list of lists of integers.
    # (actually it is very efficient dynamically resizeable
    # array)
    cdef vector[vector[int]] buckets
    
    # initialize list with b empty arrays
    for _ in xrange(b):
        buckets.push_back(vector[int]())

        
    # The code below barely changed compared to origninal
    # the only difference is the fast that we access buckets
    # slightly differently to be compliant with vector API.
    while True:
        if c_is_sorted(array):
            break
        shift = i * k

        for bucket_idx in xrange(b):
            buckets[bucket_idx].clear()
        
        for num in array:
            bucket_idx = (num >> shift) & b_m1
            buckets[bucket_idx].push_back(num)
        
        next_index = 0
        for bucket_idx in xrange(b):
            for in_bucket_idx in xrange(buckets[bucket_idx].size()):
                array[next_index] = buckets[bucket_idx][in_bucket_idx]
                next_index += 1
        
        i += 1


In [54]:
example = [5,3,2,5,6,1,2,4]
c_radix_sort(example, 16)
example

[1, 2, 2, 3, 4, 5, 5, 6]

In [56]:
example = generate_test(1000000)
cProfile.run("c_radix_sort(example, 16)")
assert is_sorted(example)

         3 function calls in 0.128 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.128    0.128 <string>:1(<module>)
        1    0.128    0.128    0.128    0.128 {_cython_magic_b8266c1b71a95275ac05a28b869d9d61.c_radix_sort}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


